
Group Members: Sinny Ye (sy544), Brian Liu (bjl229), Caio Teig (cb733)

### Introduction

`State your research questions clearly`


### Data description and cleaning

`Refer to the Phase 2 rubric for instructions on
“Datasheets for Datasets” questions. Additionally, describe the data cleaning
steps taken for your project, and provide a link to the ipynb on GitHub that
performs these data cleaning steps. If it would be helpful for understanding your
research question, feel free to include some gures from your Phase 2 EDA to
better explain your data` - helloo

### Preregistration Statement

`Provide at least 2 preregistration statements. Each statement contains a hypothesis and a description of analysis to study the hypothesis. It should also include context for why you believe in your hypothesis by referencing domain knowledge or other literature. Refer to Phase 3 rubric for more details`

**Statement #1** <br>
<u>Hypothesis:</u> VTI demonstrates a faster recovery rate than GXC following significant economic downturns, such as the COVID-19 pandemic
<br>
<br>
<u>Analysis:</u> Run a linear regression with time as the input variable (X), and cumulative returns for SPY and VTI during the recovery period after the COVID pandemic as output (Y). The slope of the regression model will indicate the recovery rate, therefore we will be testing whether BSPY > BVTIp
<br>
<br>
<u>Context:</u>


**Statement #2** <br>
<u>Hypothesis:</u> Do significant economic events such as Federal Reserve interest rate changes, the COVID-19 pandemic, and trade conflicts (2014–2024) have a more pronounced impact on SPY’s daily returns than on VTI’s, and does this effect vary across different types of events?
<br>
<br>
<u>Analysis:</u> 
- Data Analysis: Use these economic events: `Covid-19 pandemic and market crashing (2020)`, `Federal Reserve interest rate changes (2015, 2018,2022)`, `US-China Trade War (2018-2019) OR Russia-Ukraine Conflict (2022)`. <br>For each event, we will define a custom window which is 1 week before and 1 week after the event.
- Regression Model: Combine SPY and VTI returns into a single dataset with a new binary variable for the stock index (Index=1 for SPY, Index=0 for VTI)
  - Fitting the regression: Return = B0 + B1(EventIndicator) + B2(Index) + B3(EventIndicator * Index) + ϵ
  - EventIndicator: Dummy variable indicating whether a day falls within an event window (1 for within the event window, 0 otherwise)
  - Index: Dummy variable for stock index (1 for SPY, 0 for VTI)
  - EventIndicator * Index: Interaction term capturing the differential effect of events on SPY relative to VTI
 
- Interpretation: The coefficient​ B3 of the interaction term will indicate whether SPY’s daily returns are more sensitive to economic events compared to VTI. A positive and significant B3 supports the hypothesis
<br>
<u>Context:</u>
- Why Interpretation makes sense: 

### Data analysis

`Carry out the analysis for each preregistration statement.
Interpret analyses appropriately (e.g., regressions should be interpreted using the
summarizing, predicting, and outliers/oddities framework from class)`

### Evaluation of significance

`Determine statistical signicance of observed data.
Interpret ndings and conclude whether to reject or fail to reject hypotheses
outlined in preregistration statemen`ts

### Conclusions

`Summarize your answers to your opening research questions`

### Limitations

`Outline any potential harms and data gaps and describe how this
could aect conclusions made.`

### Acknowledgements and Bibliography

`Think of this as your Problem 0: what
tools, data, or resources did you use that you should cite? You may format this
section with any citation style`
